In [1]:
from main.models import Zipcode, BlockGroup, Neighborhood, Listing
from django.contrib.gis.geos import (
    GEOSGeometry, 
    Polygon, 
    MultiPolygon,
    WKBReader, 
    WKBWriter,
    Point
)
from django.contrib.gis.gdal import DataSource
import re
from lxml import etree

## Discussion regarding overlap detection of unevenly aligned geo boundaries
http://gis.stackexchange.com/questions/69139/how-do-i-use-polygon-centroids-within-a-geoqueryset

## ZCTA Import

In [2]:
# Open US Zip Code Tabulation Areas shapefile
ds = DataSource('/Users/shawn/Downloads/cb_2015_us_zcta510_500k/cb_2015_us_zcta510_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [3]:
# Create and save zipcode objects
for feature in layer:
    # Extract attrs
    root = etree.fromstring(feature.get('Description'))
    table = root.find('table')
    attrs = {
        tr[0].text: tr[1].text
        for tr in table.findall('tr')[1:]
    }
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    if geom.geom_type != 'MultiPolygon':
        geom = MultiPolygon(geom)

    # Create or update zipcode object
    Zipcode.objects.update_or_create(
        zipcode=attrs['ZCTA5CE10'],
        area_land=int(attrs['ALAND10']),
        area_water=int(attrs['AWATER10']),
        mpoly=geom
    )

## Block Group Import

In [4]:
# Open Los Angeles County Block Groups shapefile
ds = DataSource('/Users/shawn/Downloads/cb_2015_06_bg_500k/cb_2015_06_bg_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [5]:
# Create and save BlockGroup objects
for feature in layer:
    # Extract attributes
    root = etree.fromstring(feature.get('Description'))
    table = root.find('table')
    attrs = {
        tr[0].text: tr[1].text
        for tr in table.findall('tr')[1:]
    }
    
    # Extract geometry
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    if geom.geom_type != 'MultiPolygon':
        geom = MultiPolygon(geom)

    # Create or update object
    BlockGroup.objects.update_or_create(
        geoid=attrs['GEOID'], 
        area_land=int(attrs['ALAND']), 
        area_water=int(attrs['AWATER']),
        mpoly=geom
    )

In [ ]:
# TESTING

# Define custom spatial relationship
truly_overlaps = "2********" # See https://en.wikipedia.org/wiki/DE-9IM

# For each LA zipcode, show the neighborhoods it overlaps
la_zipcodes = set([
    zipcode
    for n in Neighborhood.objects.all()
    for zipcode in Zipcode.objects.filter(mpoly__relate=(n.mpoly, truly_overlaps))
])

for zipcode in set(la_zipcodes):
    print(zipcode.zipcode, Neighborhood.objects.filter(mpoly__relate=(zipcode.mpoly, truly_overlaps)), '\n')